In [18]:
import itertools
from operator import mul
from functools import reduce
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import re
import webbrowser
import os
import copy
import math



In [19]:
## CONSTANTS ##

# POSSESSIONS
OUR_POSS = 1
OPP_POSS = 2

# MOVE TYPES
OFFENSIVE = 1
DEFENSIVE = 2

# LEAGUE AVERAGES
LEAGUE_AVE_THREE_PERC = 0.362
LEAGUE_AVE_TWO_PERC = 0.510
LEAGUE_AVE_FT_PERC = 0.767

# DECREASING FACTORS
QUICK_PERC_DECREASE = 0.03 # 3% decrease in percentage if it is a quick shot

# TIME CONSUMPTION OF MOVES (IN SECS)
QUICK_TIME_CONS = (4 + 10) / 2 # The mean of the range of 4 to 10 we consider a quick shot 
SLOW_TIME_CONS = (11 + 24) / 2 # The mean of the range of 11 to 24 we consider a slow shot 
FOUL_TIME_CONS = 2




In [20]:
## IMPORT NBA LEAGUE DATA ##

team_stats = pd.read_csv('data/teamstats.csv')
team_names = team_stats["Team"]
print(team_stats)

def create_team(team_name):
    '''Given the team name it returns a Team object'''
    stats = team_stats.loc[team_stats['Team'] == team_name]
    
    three_perc = stats.iloc[0]["3P%"]
    two_perc = stats.iloc[0]["2P%"]
    ft_perc = stats.iloc[0]["FT%"]
    three_defen_perc = stats.iloc[0]["3P% Defense"]
    two_defen_perc = stats.iloc[0]["2P% Defense"]
    return Team(three_perc, two_perc, ft_perc, three_defen_perc, two_defen_perc)




                      Team    3P%    2P%    FT%  3P% Defense  2P% Defense
0            Atlanta Hawks  0.360  0.495  0.785        0.377        0.520
1           Boston Celtics  0.377  0.491  0.771        0.339        0.488
2            Brooklyn Nets  0.356  0.500  0.772        0.369        0.503
3        Charlotte Hornets  0.369  0.487  0.747        0.375        0.516
4            Chicago Bulls  0.355  0.478  0.758        0.370        0.534
5      Cleveland Cavaliers  0.372  0.540  0.779        0.368        0.533
6         Dallas Mavericks  0.360  0.496  0.763        0.363        0.526
7           Denver Nuggets  0.371  0.525  0.766        0.378        0.525
8          Detroit Pistons  0.373  0.488  0.745        0.359        0.515
9    Golden State Warriors  0.391  0.560  0.815        0.357        0.490
10         Houston Rockets  0.362  0.558  0.781        0.351        0.519
11          Indiana Pacers  0.369  0.513  0.779        0.349        0.530
12    Los Angeles Clippers  0.354  0.5

In [21]:
## Tree and Visualization Code ##

class Node:
    def __init__(self, name, parent=None):
        self.name = name
        self.parent = parent
        self.children = []
        self.optimal = False

        if parent:
            self.parent.children.append(self)
            
def convert_tree_to_dict(root, depth=8):
    '''Given the root node, converts the tree into a dictionary. Depth is the maximum depth it can get.'''
    
    d = {"name": root.name, "children": [], "optimal": root.optimal}
    if depth > 0:
        for i, child in enumerate(root.children):
            d["children"].append(convert_tree_to_dict(child, depth-1))
    return d

def show_tree(tree_root, depth=8):
    '''Given the root node, and maximum depth show the tree as an interactive webpage.'''
    
    f = open("tree.json", "w")
    f.write(json.dumps(convert_tree_to_dict(tree_root, depth)))
    f.close()
    webbrowser.open('file://' + os.path.realpath("index.html"))
    
    

In [22]:
## TEAM CLASS ##

class Team:

    def __init__(self, three_perc, two_perc, ft_perc, three_defen_perc, two_defen_perc):
        self.three_perc = three_perc
        self.two_perc = two_perc
        self.ft_perc = ft_perc
        self.three_defen_perc = three_defen_perc
        self.two_defen_perc = two_defen_perc
        
    def __str__(self):
        return "3P%: " + str(self.three_perc) + ", 2P%: " + str(self.two_perc) + ", FT%: " + str(self.ft_perc) + ", 3P% Def: " + str(self.three_defen_perc) + ", 2P% Def: " + str(self.two_defen_perc) 

    def get_two_perc_against(self, team2):
        '''Given another team calculates the two point percentage of the team.'''
        return self.two_perc * team2.two_defen_perc / LEAGUE_AVE_TWO_PERC

    def get_three_perc_against(self, team2):
        '''Given another team calculates the three point percentage of the team.'''
        return self.three_perc * team2.three_defen_perc / LEAGUE_AVE_THREE_PERC

    def get_ft_perc_against(self, team2):
        '''Given another team calculates the two point percentage of the team.'''
        return team2.ft_perc
    
    

In [23]:
## PARENT CLASS OF STATES -> CHANCE AND GAME STATES ##

class State:
    def __init__(self):
        
        # Used in Chance State
        self.probs = []
        self.states = []
        
        # Used in Game State
        self.team1 = None
        self.team2 = None
        self.score_diff = -1
        self.time = -1
        self.pos = -1
        self.pos_type = -1
        self.last_move = None

    def is_chance_state(self):
        pass

    def is_gameover(self):
        return False
    

In [24]:
## Chance State ##

# Chance States contains all the undeterministic outtcomes of a move.
# For example, if your move is to take a free throw, the chance state would contain all the Game States
# where you { make first, miss second }, { miss first, miss second }, { miss first, make second }, 
# { make first, make second } and their corresponding probabilities.

class ChanceState(State):
    def __init__(self, probs, states, move=None):
        self.probs = probs
        self.states = states

        # Initialized to dummy values (These are used in only Game State)
        self.team1 = -1
        self.team2 = -1
        self.score_diff = -1
        self.time = -1
        self.pos = -1
        self.pos_type = -1
        self.last_move = move
        
    def __str__(self):
        return str(self.last_move)

    def __eq__(self, other):
        return self.probs == other.probs and self.states == other.states

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return sum(map(lambda state: hash(state), self.states)) * 3 + int(sum(self.probs) * 5)

    def is_chance_state(self):
        return True

    def is_gameover(self):
        return False

    def get_state_prob(self, index):
        return self.probs[index]

    def get_child_states(self):
        return  self.states
    
      

In [25]:
## Game State ##

# Game States contains all the deterministic information of the state of the game.
# It stores the two teams, score difference, time, possession, possession_type and
# the last move that lead to this state.

class GameState(State):

    def __init__(self, team1, team2, score_diff, time, pos, pos_type=OFFENSIVE, move=None):
        self.team1 = team1
        self.team2 = team2
        self.score_diff = score_diff
        self.time = time
        self.pos = pos
        self.pos_type = pos_type
        self.last_move = move
        
        # Initialized to dummy values
        self.probs = []
        self.states = []


    def __eq__(self, other):
        return self.team1 == other.team1 and self.team2 == other.team2 and self.score_diff == other.score_diff and self.time == other.time and self.pos == other.pos  and self.pos_type == other.pos_type

    def __ne__(self, other):
        return not self.__eq__(other)

    def __str__(self):
        '''String representation of the state that is shown in the interactive tree visualization.'''
        
        if self.is_gameover():
            if self.score_diff > 0:
                return "We won by " + str(self.score_diff)
            elif self.score_diff == 0:
                return "We tied"
            else:
                return "We lost by " + str(-1 * self.score_diff)
        else:
            if self.score_diff > 0:
                score_text = "up by " + str(self.score_diff)
            elif self.score_diff == 0:
                score_text = "tied"
            else:
                score_text = "down by " + str(-1 * self.score_diff)
                
            if self.pos == 1:
                pos_text = "our"
            else:
                pos_text = "their"
            
          
            if self.pos_type == OFFENSIVE:
                 pos_text += " off."
            else:
                 pos_text += " def."
            
            return str(self.time) + " secs, " + score_text + ", " + pos_text + " poss."
    
    def __hash__(self):
        return self.score_diff * 3 + self.time * 5 + self.pos * 7

    def get_heuristic_score(self):
        '''Heuristic value given to the tree leaf.'''
        
        if self.score_diff <= 0:
            return 0.0
        else:
            return 1.0

    def is_gameover(self):
        return self.time <= 0

    def get_child_states(self):
        return list(map(lambda move: move.get_chance_child(), self.get_available_moves()))

    def get_available_moves(self):
        
        if self.pos_type == OFFENSIVE:
            quick_three = ThreePointer(True, self)
            slow_three = ThreePointer(False, self)
            quick_two = TwoPointer(True, self)
            slow_two = TwoPointer(False, self)
            moves = [quick_three, slow_three, quick_two, slow_two]
        elif self.pos_type == DEFENSIVE:  
            not_foul = NotFoul(self)
            foul = Foul(self)
            moves = [foul, not_foul]
        else:
            print("SHOULDN'T BE HERE!")
        
        return list(filter(lambda move: move.is_applyable(), moves))

    def is_chance_state(self):
        return False

In [26]:
## Move Class ##

# Move gets a Game State and outputs a Chance State.
# It contains numeric information that corresponds to each type of move.
# When apply is called, the Chance State is created according to these numeric description.
# time_consump : The time it takes to execute the move.
# occ_count: The number of occurance. (Free throw occurs two times for example.)
# score_change : The ammount of score change when the move deterministically occurs.
# type : Move type -> Offensive or Defensive move.
# prob_func : A function that return the prob of this move to be occured given the two teams in the Game State.
# pos_change : Bool value that the possession will change after the move.
# pos_type_change : Bool value that the possession type will change after the move.

class Move:

    def __init__(self, game_state):
        self.pos_change = True
        self.pos_type_change = True
        self.time_consump = 0
        self.score_change = 0
        self.occ_count = 0
        self.prob_func = lambda x, y: x
        self.game_state = game_state
        self.type = OFFENSIVE

    def __str__(self):
        '''String representation of the move that is shown in the interactive tree visualization.'''
        
        if (self.game_state.pos == 2 and self.score_change < 0) or (self.game_state.pos == 1 and self.score_change > 0):
            t1 = self.game_state.team1
            t2 = self.game_state.team2
        else:
            t1 = self.game_state.team2
            t2 = self.game_state.team1
        return re.sub(r"(\w)([A-Z])", r"\1 \2", self.__class__.__name__) + " (" + str(round(100 * self.prob_func(t1, t2),2)) + "%) " + " in " + str(self.time_consump) + " secs" 

    def is_applyable(self):
        '''Checks if a move can be applied.'''
        
        time = self.game_state.time - self.time_consump
        return time >= -4

    def get_chance_child(self):
        '''Applies the move to Game State and produces a Chance State'''
        
        team1 = self.game_state.team1
        team2 = self.game_state.team2

        # Possession Change
        if self.pos_change:
            new_pos = 1 if self.game_state.pos == 2 else 2
        else:
            new_pos = self.game_state.pos

        # Time Consumption
        time = self.game_state.time - self.time_consump

        score_diff = self.game_state.score_diff
        
        # Possession Type Change
        if self.pos_type_change:
            if self.type == DEFENSIVE:
                pos_type = OFFENSIVE
            else:
                pos_type = DEFENSIVE
        else:
            pos_type = self.type

        
        states = []
        probs = []

        # Picks the correct order of the teams for the score change
        if (self.game_state.pos == 2 and self.score_change < 0) or (self.game_state.pos == 1 and self.score_change > 0):
            t1 = team1
            t2 = team2
        else:
            t1 = team2
            t2 = team1
            
        prec = self.prob_func(t1, t2)
        
        # Find all combinations of make and miss
        for make_comb in itertools.product([1, 0] if prec != 1 else [1], repeat=self.occ_count):
            if self.game_state.pos == 1:
                score_diff = self.game_state.score_diff + self.score_change * sum(make_comb)
            else:
                score_diff = self.game_state.score_diff - self.score_change * sum(make_comb)

            
            # Map the binary make/miss  values to their corresponding probabilities
            prob_comb = list(map(lambda score_chan: prec if score_chan == 1 else 1.0 - prec, make_comb))
            gs = GameState(team1, team2, score_diff, time, new_pos, pos_type, self)
            states.append(gs)
            
            # Multiply the probibilities and append to the list
            probs.append(reduce(mul, prob_comb, 1))
        return ChanceState(probs, states, self)


class TwoPointer(Move):

    def __init__(self, quick, game_state):
        self.pos_change = False
        self.pos_type_change = True
        self.time_consump = QUICK_TIME_CONS if quick else SLOW_TIME_CONS
        self.score_change = 2
        self.occ_count = 1
        if quick:
            self.prob_func = lambda x, y: Team.get_two_perc_against(x,y) - QUICK_PERC_DECREASE
        else:
            self.prob_func = Team.get_two_perc_against
        self.game_state = game_state
        self.type = OFFENSIVE


class ThreePointer(Move):

    def __init__(self, quick, game_state):
        self.pos_change = False
        self.pos_type_change = True
        self.time_consump = QUICK_TIME_CONS if quick else SLOW_TIME_CONS
        self.score_change = 3
        self.occ_count = 1
        if quick:
            self.prob_func = lambda x, y: Team.get_three_perc_against(x,y) - QUICK_PERC_DECREASE
        else:
            self.prob_func = Team.get_three_perc_against
        self.game_state = game_state
        self.type = OFFENSIVE

class Foul(Move):

    def __init__(self, game_state):
        self.pos_change = True
        self.pos_type_change = False
        self.time_consump =  FOUL_TIME_CONS
        self.score_change = -1
        self.occ_count = 2
        self.prob_func = Team.get_ft_perc_against
        self.game_state = game_state
        self.type = DEFENSIVE
        
class NotFoul(Move):

    def __init__(self, game_state):
        self.pos_change = True
        self.pos_type_change = True
        self.time_consump =  0
        self.score_change = 0
        self.occ_count = 1
        self.prob_func = lambda x, y: 1
        self.game_state = game_state
        self.type = DEFENSIVE

In [27]:
## EXPECTIMINIMAX ALGORITHM ##

# ALGORITHM OUTLINE:
#
#
# function expectiminimax(node, depth)
#     if node is a terminal node or depth = 0
#         return the heuristic value of node
#     if the adversary is to play at node
#         // Return value of minimum-valued child node
#         let α := +∞
#         foreach child of node
#             α := min(α, expectiminimax(child, depth-1))
#     else if we are to play at node
#         // Return value of maximum-valued child node
#         let α := -∞
#         foreach child of node
#             α := max(α, expectiminimax(child, depth-1))
#     else if random event at node
#         // Return weighted average of all child nodes' values
#         let α := 0
#         foreach child of node
#             α := α + (Probability[child] * expectiminimax(child, depth-1))
#     return α

def run_expectiminimax(start_state, max_depth):

    dp = {}
    dp_tree = {}
    tree_root = Node(str(start_state))
    ideal_move = [""]

    
    def expectiminimax(state, depth, parent):

        if not state.is_chance_state() and (state.is_gameover() or depth == 0):
            return state.get_heuristic_score()
        nodes = []
        optimal_node_index = None
        
        if state.pos == 1:
            alpha = float('-inf')
            alpha_child = None
            for i, child in enumerate(state.get_child_states()):
                
                if child in dp:
                    # If we saw this exact node before, don't calculate it.
                    # Use the values we have saved.
                    cur_node = dp_tree[child]
                    cur_node.optimal = False
                    cur_node.parent = parent
                    parent.children.append(cur_node)
                    val = dp[child]
                else:
                    cur_node = Node(str(child), parent)
                    val = expectiminimax(child, depth - 1, cur_node)
                    dp_tree[child] = cur_node
                    dp[child] = val
                    
                nodes.append(cur_node)
                if val > alpha:
                    optimal_node_index = i
                    alpha = val
                    alpha_child = child
                    
            if optimal_node_index != None: 
                nodes[optimal_node_index].optimal = True
                
            if state == start_state:
                ideal_move[0] = str(alpha_child.last_move)
                
        elif state.pos == 2:
            alpha = float('inf')
            for i, child in enumerate(state.get_child_states()):
                
                if child in dp:
                    # If we saw this exact node before, don't calculate it.
                    # Use the values we have saved.
                    
                    cur_node = dp_tree[child]
                    cur_node.parent = parent
                    cur_node.optimal = False
                    parent.children.append(cur_node)
                    val = dp[child]
                else:
                    cur_node = Node(str(child), parent)
                    val = expectiminimax(child, depth - 1, cur_node)
                    dp_tree[child] = cur_node
                    dp[child] = val
                
                nodes.append(cur_node)
                if val < alpha:
                    optimal_node_index = i
                    alpha = val
            if optimal_node_index != None:        
                nodes[optimal_node_index].optimal = True
                    
        elif state.is_chance_state():
            alpha = 0
           
            for i, child in enumerate(state.get_child_states()):
                if child in dp:
                    cur_node = dp_tree[child]
                    cur_node.parent = parent
                    parent.children.append(cur_node)
                    val = dp[child]
                else:
                    cur_node = Node(str(child), parent)
                    val = expectiminimax(child, depth, cur_node)
                    dp_tree[child] = cur_node
                    dp[child] = val
                alpha += state.get_state_prob(i) * val
        else:
            print("SHOULDN'T BE HERE!")

        return alpha
    
    
    max_prob = expectiminimax(start_state, max_depth, tree_root)
    return ideal_move[0], max_prob, tree_root



In [28]:
cur_three_perc = .391
cur_two_perc = .660
cur_ft_perc = .75
cur_three_defen_perc = .357
cur_two_defen_perc = .490
opp_three_perc = 0.357
opp_two_perc = 0.519
opp_ft_perc = 0.804
opp_three_defen_perc = 0.366
opp_two_defen_perc = 0.534

score_diff = -1 # Points
time = 30 # Seconds left
pos = 1 # We have the ball
depth = float("inf") # Max depth the tree can go

team1 = Team(cur_three_perc, cur_two_perc, cur_ft_perc, cur_three_defen_perc, cur_two_defen_perc)
team2 = Team(opp_three_perc, opp_two_perc, opp_ft_perc, opp_three_defen_perc, opp_two_defen_perc)

game_state = GameState(team1, team2, score_diff, time, pos)
ideal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
print("The maximum expected win probability is " + str(max_prob) + ". The ideal move is to take a " + ideal_move + ".")

show_tree(tree_root, depth=15)



The maximum expected win probability is 0.46683359562. The ideal move is to take a Two Pointer (66.11%)  in 7 secs.


In [29]:
def run_expectiminimax_with_teams(team1_name, team2_name, score_diff=-1, time=30, pos=1, depth=float("inf")):
    team1 = create_team(team1_name)
    team2 = create_team(team2_name)
    start_state = GameState(team1, team2, score_diff, time, pos)
    return run_expectiminimax(start_state, depth)

In [30]:
# Takes 8.5 minutes to run
from time import gmtime, strftime
team_names = team_names[:4]
team_probs = {"Opposing Team": team_names}
for i in xrange(len(team_names)):
    team1_name = team_names[i]
    for j in xrange(len(team_names)):
        team2_name = team_names[j]
        if team1_name not in team_probs:
            team_probs[team1_name] = []
        ideal_move, max_prob, tree_root = run_expectiminimax_with_teams(team1_name, team2_name, score_diff=-1)
        team_probs[team1_name].append(str(int(100* max_prob)) + "% with " + ideal_move)
    print("Done with " + team1_name + " at " + strftime("%Y-%m-%d %H:%M:%S", gmtime()))
            
team_prob_matrix = pd.DataFrame(team_probs)
team_prob_matrix = team_prob_matrix.set_index("Opposing Team")
print(team_prob_matrix)


Done with Atlanta Hawks at 2018-05-15 14:34:06
Done with Boston Celtics at 2018-05-15 14:34:07
Done with Brooklyn Nets at 2018-05-15 14:34:09
Done with Charlotte Hornets at 2018-05-15 14:34:11
                                                Atlanta Hawks  \
Opposing Team                                                   
Atlanta Hawks      33% with Three Pointer (34.49%)  in 7 secs   
Boston Celtics     30% with Three Pointer (30.71%)  in 7 secs   
Brooklyn Nets       32% with Three Pointer (33.7%)  in 7 secs   
Charlotte Hornets  31% with Three Pointer (34.29%)  in 7 secs   

                                               Boston Celtics  \
Opposing Team                                                   
Atlanta Hawks      36% with Three Pointer (36.26%)  in 7 secs   
Boston Celtics      33% with Three Pointer (32.3%)  in 7 secs   
Brooklyn Nets      34% with Three Pointer (35.43%)  in 7 secs   
Charlotte Hornets  35% with Three Pointer (36.05%)  in 7 secs   

                         

In [31]:
# ## SENSITIVITY ANALYSIS ON TIME CONSUMPS ##


cur_three_perc = LEAGUE_AVE_THREE_PERC
cur_two_perc = LEAGUE_AVE_TWO_PERC
cur_ft_perc = LEAGUE_AVE_FT_PERC
cur_three_defen_perc = LEAGUE_AVE_THREE_PERC
cur_two_defen_perc = LEAGUE_AVE_TWO_PERC
opp_three_perc = LEAGUE_AVE_THREE_PERC
opp_two_perc = LEAGUE_AVE_TWO_PERC
opp_ft_perc = LEAGUE_AVE_FT_PERC
opp_three_defen_perc = LEAGUE_AVE_THREE_PERC
opp_two_defen_perc = LEAGUE_AVE_TWO_PERC

score_diff = -1 # Points
time = 30 # Seconds left
pos = 1 # We have the ball
depth = float("inf") # Max depth the tree can go


team1 = Team(cur_three_perc, cur_two_perc, cur_ft_perc, cur_three_defen_perc, cur_two_defen_perc)
team2 = Team(opp_three_perc, opp_two_perc, opp_ft_perc, opp_three_defen_perc, opp_two_defen_perc)

game_state = GameState(team1, team2, score_diff, time, pos)

quick_time_range = range(4,11)
slow_time_range = range(11,25)

quick_sensitivity_dict = {'''Time Consumption (secs)''': list(quick_time_range), "Optimal Move": []}

for quick_time in quick_time_range:
    QUICK_TIME_CONS = quick_time
    SLOW_TIME_CONS = (11 + 24) / 2
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    quick_sensitivity_dict["Optimal Move"].append(str(int(100 * max_prob)) + "% to win with " + optimal_move)

quick_sensitivity_table = pd.DataFrame(quick_sensitivity_dict)
quick_sensitivity_table.set_index("Time Consumption (secs)")
print("SENSITIVITY TABLE OF QUICK SHOT TIME")
print(quick_sensitivity_table)

print("\n\n")


slow_sensitivity_dict = {'''Time Consumption (secs)''': list(slow_time_range), "Optimal Move": []}

for slow_time in slow_time_range:
    SLOW_TIME_CONS = slow_time
    QUICK_TIME_CONS = (10 + 4) / 2
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    slow_sensitivity_dict["Optimal Move"].append(str(int(100 * max_prob)) + "% to win with " + optimal_move)

slow_sensitivity_table = pd.DataFrame(slow_sensitivity_dict)
slow_sensitivity_table.set_index("Time Consumption (secs)")
print("SENSITIVITY TABLE OF SLOW SHOT TIME")
print(slow_sensitivity_table)


SENSITIVITY TABLE OF QUICK SHOT TIME
                                        Optimal Move  Time Consumption (secs)
0     40% to win with Two Pointer (48.0%)  in 4 secs                        4
1     33% to win with Two Pointer (48.0%)  in 5 secs                        5
2     42% to win with Two Pointer (48.0%)  in 6 secs                        6
3   32% to win with Three Pointer (33.2%)  in 7 secs                        7
4     42% to win with Two Pointer (48.0%)  in 8 secs                        8
5   25% to win with Three Pointer (33.2%)  in 9 secs                        9
6  37% to win with Three Pointer (36.2%)  in 17 secs                       10



SENSITIVITY TABLE OF SLOW SHOT TIME
                                         Optimal Move  Time Consumption (secs)
0     46% to win with Two Pointer (51.0%)  in 11 secs                       11
1    35% to win with Three Pointer (33.2%)  in 7 secs                       12
2   32% to win with Three Pointer (36.2%)  in 13 secs          

In [32]:
# ## SENSITIVITY ANALYSIS ON GAME STARTS ##

cur_three_perc = LEAGUE_AVE_THREE_PERC
cur_two_perc = LEAGUE_AVE_TWO_PERC
cur_ft_perc = LEAGUE_AVE_FT_PERC
cur_three_defen_perc = LEAGUE_AVE_THREE_PERC
cur_two_defen_perc = LEAGUE_AVE_TWO_PERC
opp_three_perc = LEAGUE_AVE_THREE_PERC
opp_two_perc = LEAGUE_AVE_TWO_PERC
opp_ft_perc = LEAGUE_AVE_FT_PERC
opp_three_defen_perc = LEAGUE_AVE_THREE_PERC
opp_two_defen_perc = LEAGUE_AVE_TWO_PERC

score_diff = -1 # Points
time = 30 # Seconds left
pos = 1 # We have the ball
depth = float("inf") # Max depth the tree can go


team1 = Team(cur_three_perc, cur_two_perc, cur_ft_perc, cur_three_defen_perc, cur_two_defen_perc)
team2 = Team(opp_three_perc, opp_two_perc, opp_ft_perc, opp_three_defen_perc, opp_two_defen_perc)

start_time_range = range(15, 61, 3)

start_time_sensitivity_dict = {'''Start Time (secs)''': list(start_time_range), "Optimal Move": []}

for start_time in start_time_range:
    time = start_time # Seconds left
    game_state = GameState(team1, team2, score_diff, time, pos)
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    start_time_sensitivity_dict["Optimal Move"].append(str(100 * max_prob) + "% to win with " + optimal_move)

start_time_sensitivity_table = pd.DataFrame(start_time_sensitivity_dict)
start_time_sensitivity_table.set_index("Start Time (secs)")
print("SENSITIVITY TABLE OF START TIME")
print(start_time_sensitivity_table)

print("\n\n")


score_dif_time_range = range(-10,11)

score_dif_sensitivity_dict = {'''Score Difference''': list(score_dif_time_range), "Optimal Move": []}

for score in score_dif_time_range:
    score_diff = score # Seconds left
    game_state = GameState(team1, team2, score_diff, time, pos)
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    score_dif_sensitivity_dict["Optimal Move"].append(str(round(100 * max_prob, 2)) + "% to win with " + optimal_move)

score_dif_sensitivity_table = pd.DataFrame(score_dif_sensitivity_dict)
score_dif_sensitivity_table.set_index("Score Difference")
print("SENSITIVITY TABLE OF SCORE DIFFERENCE")
print(score_dif_sensitivity_table)


SENSITIVITY TABLE OF START TIME
                                         Optimal Move  Start Time (secs)
0   19.3187897993% to win with Two Pointer (48.0%)...                 15
1   48.9984% to win with Two Pointer (48.0%)  in 7...                 18
2   51.0% to win with Two Pointer (51.0%)  in 24 secs                 21
3   51.0% to win with Two Pointer (51.0%)  in 24 secs                 24
4   32.9724581064% to win with Two Pointer (48.0%)...                 27
5   36.2017724345% to win with Two Pointer (48.0%)...                 30
6   31.3878586328% to win with Three Pointer (36.2...                 33
7   34.5074021176% to win with Three Pointer (36.2...                 36
8   47.7755733839% to win with Two Pointer (51.0%)...                 39
9   31.5969503112% to win with Three Pointer (36.2...                 42
10  30.0721431514% to win with Two Pointer (48.0%)...                 45
11  46.2935783933% to win with Two Pointer (48.0%)...                 48
12  39.6820926651% 

In [33]:
## SENSITIVITY ANALYSIS ON THREE POINT SHOOTING PERCENTAGE ##

cur_three_perc = LEAGUE_AVE_THREE_PERC
cur_two_perc = LEAGUE_AVE_TWO_PERC
cur_ft_perc = LEAGUE_AVE_FT_PERC
cur_three_defen_perc = LEAGUE_AVE_THREE_PERC
cur_two_defen_perc = LEAGUE_AVE_TWO_PERC
opp_three_perc = LEAGUE_AVE_THREE_PERC
opp_two_perc = LEAGUE_AVE_TWO_PERC
opp_ft_perc = LEAGUE_AVE_FT_PERC
opp_three_defen_perc = LEAGUE_AVE_THREE_PERC
opp_two_defen_perc = LEAGUE_AVE_TWO_PERC

three_perc_change_range = range(-15, 25)
three_percs = [(delta / 100.0) + LEAGUE_AVE_THREE_PERC for delta in three_perc_change_range]
three_perc_sensitivity_dict = {'''Three Percentage''': three_percs, "Optimal Move": []}

score_diff = -1 # Points
time = 30 # Seconds left
pos = 1 # We have the ball
depth = float("inf") # Max depth the tree can go

for three_perc in three_percs:
    cur_three_perc = math.sqrt(three_perc)
    opp_three_defen_perc = math.sqrt(three_perc) * LEAGUE_AVE_THREE_PERC
    team1 = Team(cur_three_perc, cur_two_perc, cur_ft_perc, cur_three_defen_perc, cur_two_defen_perc)
    team2 = Team(opp_three_perc, opp_two_perc, opp_ft_perc, opp_three_defen_perc, opp_two_defen_perc)

    game_state = GameState(team1, team2, score_diff, time, pos)
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    three_perc_sensitivity_dict["Optimal Move"].append(str(round(100 * max_prob, 2)) + "% to win with " + optimal_move)

three_perc_sensitivity_table = pd.DataFrame(three_perc_sensitivity_dict)
three_perc_sensitivity_table.set_index("Three Percentage")
print("SENSITIVITY TABLE OF THREE PERCENTAGE")
print(three_perc_sensitivity_table)

print("\n\n")


SENSITIVITY TABLE OF THREE PERCENTAGE
                                         Optimal Move  Three Percentage
0   31.94% to win with Two Pointer (48.0%)  in 7 secs             0.212
1   32.22% to win with Two Pointer (48.0%)  in 7 secs             0.222
2   32.53% to win with Two Pointer (48.0%)  in 7 secs             0.232
3   32.85% to win with Two Pointer (48.0%)  in 7 secs             0.242
4   33.18% to win with Two Pointer (48.0%)  in 7 secs             0.252
5    33.5% to win with Two Pointer (48.0%)  in 7 secs             0.262
6   33.82% to win with Two Pointer (48.0%)  in 7 secs             0.272
7   34.14% to win with Two Pointer (48.0%)  in 7 secs             0.282
8   34.46% to win with Two Pointer (48.0%)  in 7 secs             0.292
9   34.77% to win with Two Pointer (48.0%)  in 7 secs             0.302
10  35.08% to win with Two Pointer (48.0%)  in 7 secs             0.312
11   35.3% to win with Two Pointer (48.0%)  in 7 secs             0.322
12  35.52% to win with Two

In [34]:
## SENSITIVITY ANALYSIS ON TWO POINT SHOOTING PERCENTAGE ##

cur_three_perc = LEAGUE_AVE_THREE_PERC
cur_two_perc = LEAGUE_AVE_TWO_PERC
cur_ft_perc = LEAGUE_AVE_FT_PERC
cur_three_defen_perc = LEAGUE_AVE_THREE_PERC
cur_two_defen_perc = LEAGUE_AVE_TWO_PERC
opp_three_perc = LEAGUE_AVE_THREE_PERC
opp_two_perc = LEAGUE_AVE_TWO_PERC
opp_ft_perc = LEAGUE_AVE_FT_PERC
opp_three_defen_perc = LEAGUE_AVE_THREE_PERC
opp_two_defen_perc = LEAGUE_AVE_TWO_PERC

two_perc_change_range = range(-15, 25)
two_percs = [(delta / 100.0) + LEAGUE_AVE_TWO_PERC for delta in two_perc_change_range]
two_perc_sensitivity_dict = {'''Two Percentage''': two_percs, "Optimal Move": []}

score_diff = -1 # Points
time = 30 # Seconds left
pos = 1 # We have the ball
depth = float("inf") # Max depth the tree can go

for two_perc in two_percs:
    cur_two_perc = math.sqrt(two_perc)
    opp_two_defen_perc = math.sqrt(two_perc) * LEAGUE_AVE_TWO_PERC
    team1 = Team(cur_three_perc, cur_two_perc, cur_ft_perc, cur_three_defen_perc, cur_two_defen_perc)
    team2 = Team(opp_three_perc, opp_two_perc, opp_ft_perc, opp_three_defen_perc, opp_two_defen_perc)

    game_state = GameState(team1, team2, score_diff, time, pos)
    optimal_move, max_prob, tree_root = run_expectiminimax(game_state, depth)
    two_perc_sensitivity_dict["Optimal Move"].append(str(round(100 * max_prob, 2)) + "% to win with " + optimal_move)

two_perc_sensitivity_table = pd.DataFrame(two_perc_sensitivity_dict)
two_perc_sensitivity_table.set_index("Two Percentage")
print("SENSITIVITY TABLE OF TWO PERCENTAGE")
print(two_perc_sensitivity_table)

print("\n\n")



SENSITIVITY TABLE OF TWO PERCENTAGE
                                         Optimal Move  Two Percentage
0   30.98% to win with Three Pointer (33.2%)  in 7...            0.36
1   31.11% to win with Three Pointer (33.2%)  in 7...            0.37
2   31.27% to win with Three Pointer (33.2%)  in 7...            0.38
3   31.42% to win with Three Pointer (33.2%)  in 7...            0.39
4   31.64% to win with Three Pointer (36.2%)  in 2...            0.40
5   31.88% to win with Three Pointer (36.2%)  in 2...            0.41
6   32.17% to win with Three Pointer (33.2%)  in 7...            0.42
7   32.46% to win with Three Pointer (33.2%)  in 7...            0.43
8   32.79% to win with Three Pointer (33.2%)  in 7...            0.44
9   33.2% to win with Three Pointer (33.2%)  in 7 ...            0.45
10  33.63% to win with Three Pointer (33.2%)  in 7...            0.46
11  34.07% to win with Three Pointer (33.2%)  in 7...            0.47
12  34.56% to win with Three Pointer (33.2%)  in 7... 